# Restaurant Entity-Relation Neo4j Knowledge Graph

## Cell 1: Data Loading and Initial Setup
Bu cell'de gerekli kütüphaneler import edilir ve restaurant verileri CSV dosyasından yüklenir. İlk 250 satır seçilerek analiz için hazırlanır.


In [1]:
import pandas as pd
import json
import re
import datetime

data = pd.read_csv('/Users/Serra/Desktop/bitirme/kullanılan csvler/Son_Data_Score.csv')

In [2]:
data.head()

,Mekan_Adı,Ilce,Il,Total_Weighted_Score,Tat_Score,Hizmet_Score,Ortam_Score,Fiyat-Performans_Score,Menü Çeşitliliği_Score,Temizlik_Score,Özet,Entity
0,01_adana_durumluk,Etimesgut,Ankara,0.028966,0.755000,0.225000,0.850000,0.628571,NaN,1.000000,"Restoran, özellikle Adana dürüm ve kebaplarıyl...","{""restaurant"": ""01_adana_durumluk"", ""language""..."
1,01_adanali_ismail_ustanin_yeri,Çankaya,Ankara,0.160363,0.748000,0.492188,0.718000,0.236667,0.392105,0.470000,"Bu restoran, özellikle **lezzetli Adana kebabı...","{""restaurant"": ""01_adanali_ismail_ustanin_yeri..."
2,06_ankara_kokorec,Pursaklar,Ankara,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,"Bu restoran, özellikle kokoreç konusunda öne ç...","{""restaurant"": ""06_ankara_kokorec"", ""language""..."
3,100_more_alsancak,Konak,İzmir,0.183239,0.764706,0.730000,0.851724,0.296875,0.700000,0.955556,"%100&MORE Alsancak, İzmir'in popüler mekanları...","{\n ""restaurant"": ""100_more_alsancak"",\n ""la..."
4,10_numara_restaurant,Çatalca,İstanbul,0.135506,0.827778,0.821739,0.864000,0.559091,0.960000,1.000000,"Restoran, samimi ve sıcak bir ortamıyla öne çı...","{\n ""restaurant"": ""10_numara_restaurant"",\n ..."


## Cell 3: Utility Functions and Data Processing Helpers
Bu cell'de veri işleme için yardımcı fonksiyonlar tanımlanır:
- JSON parsing fonksiyonları
- Text normalization fonksiyonları 
- Aspect mapping sözlükleri (Tat→TASTE, Hizmet→SERVICE, vb.)
- Sentiment analizi için keyword setleri
- Menü item tespiti için regex pattern'ları


In [3]:
# ========= 1) Yardımcılar =========
import re, json, ast, datetime
import pandas as pd
import unicodedata

def now_utc_iso():
    return datetime.datetime.now(datetime.timezone.utc).isoformat()

def norm(s: str) -> str:
    """Normalize plain strings, Türkçe karakter uyumlu"""
    if s is None:
        return ""
    s = str(s).strip()
    s = unicodedata.normalize("NFC", s)  # Unicode normalize
    # Türkçe özel case map
    tr_map = str.maketrans({"I": "ı", "İ": "i"})
    s = s.translate(tr_map).casefold()
    return s

def normalize_surface(s: str) -> str:
    """Normalize for surface names (menu, aspect), Türkçe karakter uyumlu"""
    s = (s or "").strip()
    s = unicodedata.normalize("NFC", s)
    tr_map = str.maketrans({"I": "ı", "İ": "i"})
    s = s.translate(tr_map).casefold()
    s = re.sub(r"[^\w\sçğıöşü-]+", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def safe_parse_summary(raw):
    if raw is None or (isinstance(raw, float) and pd.isna(raw)):
        return None
    if isinstance(raw, dict):
        return raw
    if isinstance(raw, (bytes, bytearray)):
        raw = raw.decode("utf-8", "ignore")
    s = str(raw).strip()
    if not s:
        return None
    s = s.lstrip("\ufeff").replace("“","\"").replace("”","\"").replace("’","'").replace("‘","'")
    try:
        return json.loads(s)
    except:
        pass
    if "{" in s and "}" in s:
        core = s[s.find("{"): s.rfind("}")+1]
        try:
            return json.loads(core)
        except:
            core2 = re.sub(r",\s*([}\]])", r"\1", core)
            try:
                return json.loads(core2)
            except:
                pass
    try:
        obj = ast.literal_eval(s)
        if isinstance(obj, dict):
            return obj
    except:
        pass
    return None

# ========= 2) Sözlükler / Haritalar =========
ASPECT_MAP = {
  "Tat":"TASTE","Hizmet":"SERVICE","Ortam":"AMBIENCE",
  "Fiyat-Performans":"VALUE","Menü Çeşitliliği":"MENU_VARIETY","Temizlik":"CLEANLINESS"
}

NEG_HINTS = {"değil","yetersiz","küçük","pahalı","kirli","eksik","sorun", "sorunlu","sıkıntı"}
POS_HINTS = {"lezzetli","doyurucu","uygun fiyat","güler yüzlü","temiz"}

AMENITIES = {"otopark","çocuk parkı","bahçe","deniz manzarası","canlı müzik","alkolsüz","teras"}
OCCASIONS = {"romantik","aile","sakin","doğum günü","iş yemeği", "eğlenceli"}

# Menü isimleri (çekirdek; kendi KB’inle genişlet)
MENU_LEX = {
    "adana dürüm","kebap","lahmacun","pide","köfte","döner","balık","çorba","salata",
    "meze","tatlı","baklava","künefe","sushi","ramen","pizza","hamburger","burger",
    "biftek","mantı","falafel","humus","shawarma","taco","burrito","curry",
    "penne","spaghetti","risotto","gnocchi","türk mutfağı", "geleneksel türk mutfağı"
}
MENU_PAT = re.compile(
    r"\b(kebap|dürüm|döner|pide|lahmacun|köfte|mantı|balık|çorba|salata|meze|tatlı|"
    r"baklava|künefe|sushi|ramen|pizza|hamburger|burger|biftek|falafel|humus|shawarma|"
    r"taco|burrito|curry|penne|spaghetti|risotto|gnocchi)s?\b",
    flags=re.IGNORECASE
)

def sentiment_from(polarity, spans):
    text = " ".join(spans or []).lower()
    if any(w in text for w in NEG_HINTS): return "negative"
    if any(w in text for w in POS_HINTS): return "positive"
    if polarity == "+": return "positive"
    if polarity == "-": return "negative"
    return "neutral"

def is_menu_item(term: str) -> bool:
    t = normalize_surface(term)
    return (t in MENU_LEX) or bool(MENU_PAT.search(t))

REL_LOCAL_MAP = {
    "co-occur": "CO_OCCUR",
    "co_occur": "CO_OCCUR",
    "contrast": "CONTRASTS",
    "contrasts": "CONTRASTS",
    "boosts": "BOOSTS",
    "implies": "IMPLIES",
    "affects_negatively": "AFFECTS_NEGATIVELY"
}

## Cell 4: Knowledge Graph Edge Builder
Bu cell'de restaurant verilerinden knowledge graph edge'leri oluşturan ana fonksiyon tanımlanır:
- RATED_FOR: Restaurant'ların aspect'lere göre değerlendirmeleri
- SERVES: Restaurant'ların servis ettiği menü itemları
- HAS_AMENITY: Restaurant'ların sahip olduğu imkanlar (otopark, bahçe, vb.)
- SUITS_OCCASION: Restaurant'ların uygun olduğu durumlar (romantik, aile, vb.)
- LOCATED_IN: Restaurant'ların konum bilgileri (il/ilçe)


- Bu fonksiyon bir restoran dokümanından (doc) graph edge’leri üretir. Amaç, restoranı farklı özellikler, menü öğeleri, imkânlar, durumlar ve lokasyonlarla ilişkilendirmektir.
Temel yapı
- Restoran adı normalize edilerek restaurant:<name> node’u oluşturuluyor.
- entities içinden kategorilere göre (ASPECT_MAP) geziyor.
- Menü öğeleri → SERVES ilişkisi.
- Tat/diğer aspektler → RATED_FOR ilişkisi (sentiment, score, confidence gibi props ile).
- Ek taramalar
- Tüm evidence span’ler birleştirilip metin içinde AMENITIES ve OCCASIONS aranıyor. Bulunursa HAS_AMENITY ve SUITS_OCCASION ilişkileri ekleniyor.
- İl/ilçe parametreleri varsa → LOCATED_IN ilişkileri oluşturuluyor.
- relations_local varsa → aspect→aspect ilişkileri (ör. “better_than”) ekleniyor.
- Özellikler
- Her edge’de props alanı var: sentiment, score, confidence, evidence, source, updated_at.
- Score = 0.5salience + 0.5confidence.
- Tarihler now_utc_iso() ile güncel ekleniyor.

In [4]:
def build_edges(doc, source="llm_entity_json", il=None, ilce=None):
    rest = f"restaurant:{norm(doc.get('restaurant'))}"
    edges = []
    entities = doc.get("entities") or {}

    for k, items in entities.items():
        group = ASPECT_MAP.get(k)
        if not group:
            continue
        for it in items:
            name = (it.get("normalize_to") or it.get("name") or "").strip()
            if not name:
                continue
            spans = it.get("evidence_spans", []) or []
            sent = sentiment_from(it.get("polarity"), spans)
            sal = float(it.get("salience", 0.5))
            conf = float(it.get("confidence", 0.5))
            score = round(0.5*sal + 0.5*conf, 3)

            if group == "TASTE":
                # Menü ise: SERVES yaz, RATED_FOR yazma
                if is_menu_item(name):
                    edges.append({
                        "head": rest, "relation": "SERVES",
                        "tail": f"menu:{normalize_surface(name)}",
                        "props": {
                            "confidence": min(1.0, conf + 0.1),
                            "evidence_spans": spans, "source": source, "updated_at": now_utc_iso()
                        }
                    })
                    continue
                # Menü değilse gerçek TASTE aspekti → RATED_FOR
                edges.append({
                    "head": rest, "relation": "RATED_FOR",
                    "tail": f"aspect:{normalize_surface(name)}",
                    "props": {
                        "dimension": "TASTE", "sentiment": sent, "score": score,
                        "confidence": conf, "evidence_spans": spans, "votes": 1,
                        "source": source, "updated_at": now_utc_iso()
                    }
                })
                continue

            # Diğer gruplar: RATED_FOR
            edges.append({
                "head": rest, "relation": "RATED_FOR",
                "tail": f"aspect:{normalize_surface(name)}",
                "props": {
                    "dimension": group, "sentiment": sent, "score": score,
                    "confidence": conf, "evidence_spans": spans, "votes": 1,
                    "source": source, "updated_at": now_utc_iso()
                }
            })

    # Amenity & Occasion (span araması)
    all_text = " ".join([
        " ".join(e.get("evidence_spans", []) or [])
        for arr in entities.values() for e in arr
    ]).lower()

    for amen in AMENITIES:
        if amen in all_text:
            edges.append({
                "head": rest, "relation": "HAS_AMENITY",
                "tail": f"amenity:{amen}",
                "props": {
                    "availability": "true", "confidence": 0.8,
                    "evidence_spans": [amen], "source": source, "updated_at": now_utc_iso()
                }
            })

    for occ in OCCASIONS:
        if occ in all_text:
            edges.append({
                "head": rest, "relation": "SUITS_OCCASION",
                "tail": f"occasion:{occ}",
                "props": {
                    "strength": "medium", "confidence": 0.75,
                    "evidence_spans": [occ], "source": source, "updated_at": now_utc_iso()
                }
            })

    # LOCATED_IN: İl / İlçe kolonlarından
    if ilce and str(ilce).strip():
        district_name = norm(str(ilce))
        edges.append({
            "head": rest, "relation": "LOCATED_IN",
            "tail": f"location:{district_name}",
            "props": {
                "level": "district",
                "name": district_name,   # 🔹 name eklendi
                "confidence": 1.0,
                "evidence_spans": [],
                "source": source,
                "updated_at": now_utc_iso()
            }
        })
    if il and str(il).strip():
        city_name = norm(str(il))
        edges.append({
            "head": rest, "relation": "LOCATED_IN",
            "tail": f"location:{city_name}",
            "props": {
                "level": "city",
                "name": city_name,       # 🔹 name eklendi
                "confidence": 1.0,
                "evidence_spans": [],
                "source": source,
                "updated_at": now_utc_iso()
            }
        })
        
            # 3.5 NEW: relations_local → edges
    for lr in (doc.get("relations_local") or []):
        rtype = REL_LOCAL_MAP.get(str(lr.get("type", "")).lower())
        if not rtype:
            continue
        a = lr.get("a"); b = lr.get("b")
        if not a or not b:
            continue
        edges.append({
            "head": f"aspect:{normalize_surface(a)}",
            "relation": rtype,
            "tail": f"aspect:{normalize_surface(b)}",
            "props": {
                "strength": lr.get("strength"),
                "source": source, "updated_at": now_utc_iso()
            }
        })

    return edges

## Cell 5: Edge Generation and DataFrame Creation
Bu cell'de tüm DataFrame satırları üzerinde döngü yapılarak edge'ler oluşturulur:
- Her satırdaki JSON verisi parse edilir
- build_edges fonksiyonu çağırılarak ilişkiler çıkarılır
- Tüm edge'ler tek bir DataFrame'de toplanır
- LOCATED_IN edge'lerinin sayısı kontrol edilir (498 adet)


In [39]:
# ========= 4) DataFrame üzerinde çalıştır =========
city_col     = "Il"    # İl için kolon adı
district_col = "Ilce"  # İlçe için kolon adı

all_edges = []
for idx, row in data.iterrows():
    try:
        doc = safe_parse_summary(row.get("Entity"))
        if doc is None:
            continue

        il   = row.get(city_col)      # "Sehir"
        ilce = row.get(district_col)  # "Ilce"

        edges = build_edges(doc, source=f"row_{idx}", il=il, ilce=ilce)
        for e in edges:
            e["doc_index"] = idx
            all_edges.append(e)
    except Exception as ex:
        print(f"[ERROR] row {idx}: {ex}")

edges_df = pd.DataFrame(all_edges)

# Hızlı kontrol:
print("LOCATED_IN edge sayısı:", (edges_df["relation"] == "LOCATED_IN").sum())
print(edges_df[edges_df["relation"] == "LOCATED_IN"].head(20))

LOCATED_IN edge sayısı: 4670
                                                  head    relation  \
12                        restaurant:01_adana_durumluk  LOCATED_IN   
13                        restaurant:01_adana_durumluk  LOCATED_IN   
32           restaurant:01_adanali_ismail_ustanin_yeri  LOCATED_IN   
33           restaurant:01_adanali_ismail_ustanin_yeri  LOCATED_IN   
48                        restaurant:06_ankara_kokorec  LOCATED_IN   
49                        restaurant:06_ankara_kokorec  LOCATED_IN   
69                        restaurant:100_more_alsancak  LOCATED_IN   
70                        restaurant:100_more_alsancak  LOCATED_IN   
86                     restaurant:10_numara_restaurant  LOCATED_IN   
87                     restaurant:10_numara_restaurant  LOCATED_IN   
115                   restaurant:1453_osmanli_bostanli  LOCATED_IN   
116                   restaurant:1453_osmanli_bostanli  LOCATED_IN   
141  restaurant:16_roof_swissotel_the_bosphorus_ist...  LOCAT

## Cell 6: Edge DataFrame Display
Bu cell'de oluşturulan edges_df DataFrame'i görüntülenir. Toplam 4522 satır ve 5 kolon (head, relation, tail, props, doc_index) içerir.


In [6]:
edges_df

,head,relation,tail,props,doc_index
0,restaurant:01_adana_durumluk,RATED_FOR,aspect:lezzet,"{'dimension': 'TASTE', 'sentiment': 'positive'...",0
1,restaurant:01_adana_durumluk,SERVES,menu:adana dürüm,"{'confidence': 1.0, 'evidence_spans': ['Adana ...",0
2,restaurant:01_adana_durumluk,SERVES,menu:kebap,"{'confidence': 1.0, 'evidence_spans': ['Adana ...",0
3,restaurant:01_adana_durumluk,RATED_FOR,aspect:çiğköfte ikramı,"{'dimension': 'TASTE', 'sentiment': 'negative'...",0
4,restaurant:01_adana_durumluk,RATED_FOR,aspect:güler yüzlü hizmet,"{'dimension': 'SERVICE', 'sentiment': 'positiv...",0
...,...,...,...,...,...
49046,restaurant:ziyafet_restaurant_catering,RATED_FOR,aspect:sinek,"{'dimension': 'CLEANLINESS', 'sentiment': 'neg...",2364
49047,restaurant:ziyafet_restaurant_catering,RATED_FOR,aspect:kıl,"{'dimension': 'CLEANLINESS', 'sentiment': 'neg...",2364
49048,restaurant:ziyafet_restaurant_catering,LOCATED_IN,location:sincan,"{'level': 'district', 'name': 'sincan', 'confi...",2364
49049,restaurant:ziyafet_restaurant_catering,LOCATED_IN,location:ankara,"{'level': 'city', 'name': 'ankara', 'confidenc...",2364


## Cell 7: Relation Type Statistics
Bu cell'de edge türlerinin dağılımı gösterilir:
- RATED_FOR: 3601 (en fazla - restaurant değerlendirmeleri)
- LOCATED_IN: 498 (konum bilgileri)
- SERVES: 316 (menü itemları)
- SUITS_OCCASION: 63 (durum uygunlukları)
- HAS_AMENITY: 44 (imkanlar)


In [7]:
edges_df["relation"].value_counts() 

relation
RATED_FOR         36709
LOCATED_IN         4670
CO_OCCUR           3490
SERVES             2961
SUITS_OCCASION      622
HAS_AMENITY         442
CONTRASTS           153
IMPLIES               4
Name: count, dtype: int64

## Cell 9: Neo4j Package Installation
Bu cell'de Neo4j ve pandas kütüphaneleri pip ile yüklenir. Gerekli dependency'ler kontrol edilir.

In [8]:
# Install required packages for Neo4j
# Run this cell first to install dependencies
%pip install neo4j pandas

Note: you may need to restart the kernel to use updated packages.


## Cell 10: Neo4j Knowledge Graph Class Definition
Bu cell'de Neo4j veritabanına bağlanmak ve knowledge graph oluşturmak için comprehensive bir class tanımlanır:

**Temel özellikler:**
- Neo4j bağlantı yönetimi
- Veritabanı temizleme
- Constraint ve index oluşturma
- Node ve relationship oluşturma
- Bulk import fonksiyonları
- İstatistik alma

**Node türleri:** Restaurant, Aspect, Location, Amenity, Occasion, MenuItem
**Relationship türleri:** RATED_FOR, SERVES, LOCATED_IN, HAS_AMENITY, SUITS_OCCASION


In [9]:
# ========= 5) Neo4j Knowledge Graph Builder =========
from neo4j import GraphDatabase
import logging

class Neo4jKnowledgeGraph:
    def __init__(self, uri="bolt://localhost:7687", user="neo4j", password="password"):
        """
        Initialize Neo4j connection
        
        Default connection assumes local Neo4j instance:
        - URI: bolt://localhost:7687
        - User: neo4j
        - Password: password (change this!)
        
        Make sure Neo4j is running before using this class.
        """
        self.driver = GraphDatabase.driver(uri, auth=(user, password))
        
    def close(self):
        self.driver.close()
        
    def clear_database(self):
        """Clear all nodes and relationships - USE WITH CAUTION!"""
        with self.driver.session() as session:
            session.run("MATCH (n) DETACH DELETE n")
            print("Database cleared!")
            
    def create_constraints_and_indexes(self):
        """Create constraints and indexes for better performance"""
        with self.driver.session() as session:
            constraints_and_indexes = [
                # Constraints (ensure uniqueness)
                "CREATE CONSTRAINT restaurant_id IF NOT EXISTS FOR (r:Restaurant) REQUIRE r.id IS UNIQUE",
                "CREATE CONSTRAINT aspect_id IF NOT EXISTS FOR (a:Aspect) REQUIRE a.id IS UNIQUE", 
                "CREATE CONSTRAINT location_id IF NOT EXISTS FOR (l:Location) REQUIRE l.id IS UNIQUE",
                "CREATE CONSTRAINT amenity_id IF NOT EXISTS FOR (am:Amenity) REQUIRE am.id IS UNIQUE",
                "CREATE CONSTRAINT occasion_id IF NOT EXISTS FOR (o:Occasion) REQUIRE o.id IS UNIQUE",
                "CREATE CONSTRAINT menu_id IF NOT EXISTS FOR (m:MenuItem) REQUIRE m.id IS UNIQUE",
                
                # Indexes for faster queries
                "CREATE INDEX restaurant_name IF NOT EXISTS FOR (r:Restaurant) ON (r.name)",
                "CREATE INDEX location_name IF NOT EXISTS FOR (l:Location) ON (l.name)",
            ]
            
            for cmd in constraints_and_indexes:
                try:
                    session.run(cmd)
                    print(f"✓ {cmd.split()[1]} created")
                except Exception as e:
                    print(f"⚠ {cmd.split()[1]} already exists or error: {e}")
                    
    def extract_node_info(self, node_id):
        """Extract node type and properties from node_id like 'restaurant:name' or 'aspect:taste'"""
        if ":" not in node_id:
            return "Unknown", {"id": node_id, "name": node_id}
            
        node_type, name = node_id.split(":", 1)
        
        # Map node types to Neo4j labels
        label_map = {
            "restaurant": "Restaurant",
            "aspect": "Aspect", 
            "location": "Location",
            "amenity": "Amenity",
            "occasion": "Occasion",
            "menu": "MenuItem"
        }
        
        label = label_map.get(node_type, "Entity")
        props = {"id": node_id, "name": name, "type": node_type}
        
        return label, props
        
    def get_stats(self):
        """Get basic statistics about the knowledge graph"""
        with self.driver.session() as session:
            # Count nodes by label
            node_counts = {}
            labels_result = session.run("CALL db.labels()")
            for record in labels_result:
                label = record[0]
                count_result = session.run(f"MATCH (n:{label}) RETURN count(n) as count")
                node_counts[label] = count_result.single()['count']
            
            # Count relationships by type
            rel_counts = {}
            rel_types_result = session.run("CALL db.relationshipTypes()")
            for record in rel_types_result:
                rel_type = record[0]
                count_result = session.run(f"MATCH ()-[r:{rel_type}]->() RETURN count(r) as count")
                rel_counts[rel_type] = count_result.single()['count']
            
            return {
                'nodes': node_counts,
                'relationships': rel_counts,
                'total_nodes': sum(node_counts.values()),
                'total_relationships': sum(rel_counts.values())
            }
            
    def create_node(self, node_id, additional_props=None):
        """Create a single node in Neo4j"""
        label, props = self.extract_node_info(node_id)
        
        if additional_props:
            props.update(additional_props)
            
        with self.driver.session() as session:
            query = f"""
            MERGE (n:{label} {{id: $id}})
            SET n += $props
            RETURN n
            """
            session.run(query, id=node_id, props=props)
            
    def create_relationship(self, head_id, relation, tail_id, props=None):
        """Create a relationship between two nodes"""
        # Ensure both nodes exist first
        self.create_node(head_id)
        self.create_node(tail_id)
        
        head_label, _ = self.extract_node_info(head_id)
        tail_label, _ = self.extract_node_info(tail_id)
        
        props = props or {}
        
        with self.driver.session() as session:
            query = f"""
            MATCH (h:{head_label} {{id: $head_id}})
            MATCH (t:{tail_label} {{id: $tail_id}})
            MERGE (h)-[r:{relation}]->(t)
            SET r += $props
            RETURN r
            """
            session.run(query, head_id=head_id, tail_id=tail_id, props=props)
            
    def bulk_import_edges(self, edges_df, batch_size=1000):
        """Efficiently import all edges from the DataFrame"""
        print(f"Importing {len(edges_df)} edges in batches of {batch_size}...")
        
        # First pass: Create all unique nodes
        unique_nodes = set()
        for _, row in edges_df.iterrows():
            unique_nodes.add(row['head'])
            unique_nodes.add(row['tail'])
            
        print(f"Creating {len(unique_nodes)} unique nodes...")
        
        with self.driver.session() as session:
            for i, node_id in enumerate(unique_nodes):
                if i % 100 == 0:
                    print(f"  Nodes created: {i}/{len(unique_nodes)}")
                    
                label, props = self.extract_node_info(node_id)
                query = f"""
                MERGE (n:{label} {{id: $id}})
                SET n += $props
                """
                session.run(query, id=node_id, props=props)
        
        print("✓ All nodes created!")
        
        # Second pass: Create relationships in batches
        print("Creating relationships...")
        
        for i in range(0, len(edges_df), batch_size):
            batch = edges_df.iloc[i:i+batch_size]
            print(f"  Processing batch {i//batch_size + 1}/{(len(edges_df)-1)//batch_size + 1}")
            
            with self.driver.session() as session:
                for _, row in batch.iterrows():
                    head_id = row['head']
                    tail_id = row['tail'] 
                    relation = row['relation']
                    props = row.get('props', {})
                    
                    # Add doc_index if available
                    if 'doc_index' in row:
                        props['doc_index'] = row['doc_index']
                    
                    head_label, _ = self.extract_node_info(head_id)
                    tail_label, _ = self.extract_node_info(tail_id)
                    
                    query = f"""
                    MATCH (h:{head_label} {{id: $head_id}})
                    MATCH (t:{tail_label} {{id: $tail_id}})
                    MERGE (h)-[r:{relation}]->(t)
                    SET r += $props
                    """
                    session.run(query, head_id=head_id, tail_id=tail_id, props=props)
        
        print("✓ All relationships created!")


## Cell 12: Neo4j Connection Test
Bu cell'de Neo4j veritabanına bağlantı test edilir:
- Neo4jKnowledgeGraph instance oluşturulur
- Bağlantı URI: neo4j://127.0.0.1:7687
- Kullanıcı: neo4j, Şifre: 12345678
- Mevcut veritabanı istatistikleri alınır (0 node, 0 relationship)

⚠️ Neo4j'nin lokal olarak çalışıyor olması gerekir!


In [10]:
# Initialize the knowledge graph
kg = Neo4jKnowledgeGraph(
    uri="neo4j://127.0.0.1:7687", 
    user="neo4j", 
    password="12345678"  # ← bunu güçlü bir şifre ile değiştir
)

try:
    # Test connection
    print("Testing Neo4j connection...")
    stats = kg.get_stats()
    print("✓ Connected to Neo4j successfully!")
    print(f"Current database has {stats['total_nodes']} nodes and {stats['total_relationships']} relationships")
    
except Exception as e:
    print(f"❌ Connection failed: {e}")
    print("\nMake sure:")
    print("1. Neo4j is running")
    print("2. The password is correct") 
    print("3. The URI is correct (default: bolt://localhost:7687)")
    kg = None


Testing Neo4j connection...
✓ Connected to Neo4j successfully!
Current database has 13386 nodes and 50183 relationships


## Cell 13: Knowledge Graph Creation and Data Import
Bu cell'de knowledge graph oluşturulur ve veriler Neo4j'ye import edilir:

**Adımlar:**
1. Database schema oluşturma (constraints ve indexes)
2. 4522 edge'i 500'lük batch'ler halinde import etme
3. 2140 unique node oluşturma
4. 4479 relationship oluşturma

**Final İstatistikler:**
- Restaurant: 249, Aspect: 1704, Location: 24, Amenity: 6, Occasion: 5, MenuItem: 152
- RATED_FOR: 3564, SERVES: 312, LOCATED_IN: 496, HAS_AMENITY: 44, SUITS_OCCASION: 63


In [11]:
# ========= 7) Import Data into Neo4j =========

if kg is not None:
    print("Setting up database schema...")
    
    # Create constraints and indexes
    kg.create_constraints_and_indexes()
    
    print(f"\nImporting {len(edges_df)} edges into Neo4j...")
    print("This may take a few minutes...")
    
    # Import all edges (this creates nodes and relationships)
    kg.bulk_import_edges(edges_df, batch_size=500)
    
    print("\n🎉 Knowledge graph created successfully!")
    
    # Get final statistics
    final_stats = kg.get_stats()
    print(f"\nFinal Statistics:")
    print(f"📊 Total Nodes: {final_stats['total_nodes']}")
    print(f"🔗 Total Relationships: {final_stats['total_relationships']}")
    print(f"\nNodes by type:")
    for label, count in final_stats['nodes'].items():
        print(f"  {label}: {count}")
    print(f"\nRelationships by type:")
    for rel_type, count in final_stats['relationships'].items():
        print(f"  {rel_type}: {count}")
        
else:
    print("❌ Skipping import - no Neo4j connection")

Setting up database schema...
✓ CONSTRAINT created
✓ CONSTRAINT created
✓ CONSTRAINT created
✓ CONSTRAINT created
✓ CONSTRAINT created
✓ CONSTRAINT created
✓ INDEX created
✓ INDEX created

Importing 49051 edges into Neo4j...
This may take a few minutes...
Importing 49051 edges in batches of 500...
Creating 13034 unique nodes...
  Nodes created: 0/13034
  Nodes created: 100/13034
  Nodes created: 200/13034
  Nodes created: 300/13034
  Nodes created: 400/13034
  Nodes created: 500/13034
  Nodes created: 600/13034
  Nodes created: 700/13034
  Nodes created: 800/13034
  Nodes created: 900/13034
  Nodes created: 1000/13034
  Nodes created: 1100/13034
  Nodes created: 1200/13034
  Nodes created: 1300/13034
  Nodes created: 1400/13034
  Nodes created: 1500/13034
  Nodes created: 1600/13034
  Nodes created: 1700/13034
  Nodes created: 1800/13034
  Nodes created: 1900/13034
  Nodes created: 2000/13034
  Nodes created: 2100/13034
  Nodes created: 2200/13034
  Nodes created: 2300/13034
  Nodes cr

In [12]:
print(data["Il"].unique())

['Ankara' 'İzmir' 'İstanbul']


In [13]:
print(edges_df[edges_df["relation"] == "LOCATED_IN"].head(10))
print(edges_df[edges_df["relation"] == "LOCATED_IN"]["tail"].unique())

                                         head    relation                tail  \
12               restaurant:01_adana_durumluk  LOCATED_IN  location:etimesgut   
13               restaurant:01_adana_durumluk  LOCATED_IN     location:ankara   
32  restaurant:01_adanali_ismail_ustanin_yeri  LOCATED_IN    location:çankaya   
33  restaurant:01_adanali_ismail_ustanin_yeri  LOCATED_IN     location:ankara   
48               restaurant:06_ankara_kokorec  LOCATED_IN  location:pursaklar   
49               restaurant:06_ankara_kokorec  LOCATED_IN     location:ankara   
69               restaurant:100_more_alsancak  LOCATED_IN      location:konak   
70               restaurant:100_more_alsancak  LOCATED_IN      location:izmir   
86            restaurant:10_numara_restaurant  LOCATED_IN    location:çatalca   
87            restaurant:10_numara_restaurant  LOCATED_IN   location:istanbul   

                                                props  doc_index  
12  {'level': 'district', 'name': 'etimes

## Cell 14: Validation Queries and Knowledge Graph Testing
Bu cell'de oluşturulan knowledge graph'in doğruluğunu test eden örnek Cypher sorguları çalıştırılır:

**Test Sorguları:**
1. **Ankara'daki restaurant'lar** - LOCATED_IN ilişkisini test eder (10 sonuç)
2. **Pozitif tat değerlendirmeleri** - RATED_FOR ilişkisini test eder (score'a göre sıralı)
3. **İmkanlara sahip restaurant'lar** - HAS_AMENITY ilişkisini test eder

Tüm sorgular başarıyla çalışır ve beklenen sonuçları döndürür ✓


In [14]:
# ========= 8) Validation and Example Queries =========
def run_sample_queries(kg):
    """Knowledge Graph üzerinde doğrulama ve örnek sorgular çalıştırır"""
    
    if kg is None:
        print("❌ Neo4j bağlantısı yok")
        return
    
    queries = [
        {
            "name": "Ankara'daki mekanlar",
            "query": """
            MATCH (r:Restaurant)-[:LOCATED_IN]->(l:Location)
            WHERE toLower(l.name) = 'ankara'
            RETURN r.name as restoran, l.name as lokasyon
            LIMIT 10
            """,
            "description": "Ankara'da bulunan restoranları gösterir"
        },
        
        {
            "name": "Olumlu tat puanı alan restoranlar",
            "query": """
            MATCH (r:Restaurant)-[rating:RATED_FOR]->(a:Aspect)
            WHERE rating.dimension = 'TASTE' AND rating.sentiment = 'positive'
            RETURN r.name as restoran, a.name as aspekt, rating.score as puan
            ORDER BY rating.score DESC
            LIMIT 10
            """,
            "description": "En yüksek olumlu tat puanına sahip restoranları listeler"
        },
        
        {
            "name": "Restoranların imkanları",
            "query": """
            MATCH (r:Restaurant)-[:HAS_AMENITY]->(am:Amenity)
            RETURN r.name as restoran, collect(am.name) as imkanlar
            LIMIT 10
            """,
            "description": "Restoranları ve sahip oldukları imkanları gösterir"
        },
        
        {
            "name": "İstanbul'da romantik ve yüksek tat puanı olan restoranlar",
            "query": """
            MATCH (r:Restaurant)-[:LOCATED_IN]->(l:Location)
            MATCH (r)-[rating:RATED_FOR]->(a:Aspect)
            MATCH (r)-[:SUITS_OCCASION]->(o:Occasion)
            WHERE toLower(l.name) = 'istanbul'
              AND rating.dimension = 'TASTE'
              AND rating.sentiment = 'positive'
              AND toLower(o.name) = 'romantik'
            RETURN r.name as restoran, avg(rating.score) as ortalama_puan
            ORDER BY ortalama_puan DESC
            LIMIT 10
            """,
            "description": "İstanbul'daki romantik restoranlardan yüksek tat puanı alanları listeler"
        }
    ]
    
    print("🔍 Doğrulama sorguları çalıştırılıyor...\n")
    
    with kg.driver.session() as session:
        for i, q in enumerate(queries, 1):
            print(f"{i}. {q['name']}")
            print(f"   {q['description']}")
            
            try:
                result = session.run(q['query'])
                records = list(result)
                
                if records:
                    print(f"   ✓ {len(records)} sonuç bulundu:")
                    for j, record in enumerate(records[:3]):  # İlk 3 sonucu göster
                        print(f"     {j+1}. {dict(record)}")
                    if len(records) > 3:
                        print(f"     ... ve {len(records) - 3} tane daha")
                else:
                    print("   ⚠ Sonuç bulunamadı")
                    
            except Exception as e:
                print(f"   ❌ Sorgu hatası: {e}")
            
            print()

# Çalıştır
run_sample_queries(kg)



🔍 Doğrulama sorguları çalıştırılıyor...

1. Ankara'daki mekanlar
   Ankara'da bulunan restoranları gösterir
   ✓ 10 sonuç bulundu:
     1. {'restoran': 'beyzade_cafe_elmadag', 'lokasyon': 'ankara'}
     2. {'restoran': 'bolu_mangal_evi', 'lokasyon': 'ankara'}
     3. {'restoran': 'cafemodern', 'lokasyon': 'ankara'}
     ... ve 7 tane daha

2. Olumlu tat puanı alan restoranlar
   En yüksek olumlu tat puanına sahip restoranları listeler
   ✓ 10 sonuç bulundu:
     1. {'restoran': 'hakki_ustanin_yeri_yemek_sanayi', 'aspekt': 'bağımlılık yapan lezzet', 'puan': 0.98}
     2. {'restoran': 'biyesen', 'aspekt': 'türk köftesi', 'puan': 0.975}
     3. {'restoran': 'are_pizza_simit_cafe_fast_food', 'aspekt': 'lezzet', 'puan': 0.97}
     ... ve 7 tane daha

3. Restoranların imkanları
   Restoranları ve sahip oldukları imkanları gösterir
   ✓ 10 sonuç bulundu:
     1. {'restoran': 'bahceli_cafe_restaurant', 'imkanlar': ['deniz manzarası', 'bahçe', 'canlı müzik']}
     2. {'restoran': 'big_chefs_kiy

# Burası deneme içindi

In [15]:
def debug_istanbul_romantik_taste(kg):
    if kg is None:
        print("❌ Neo4j bağlantısı yok")
        return
    
    with kg.driver.session() as session:
        print("🔎 Debug başlıyor...\n")

        # 1) İstanbul location kontrolü
        q1 = """
        MATCH (l:Location)
        WHERE toLower(l.name) = 'istanbul'
        RETURN l.name as name, l.id as id
        """
        res1 = list(session.run(q1))
        print("1) Location kontrolü:")
        if res1:
            print(f"   ✓ İstanbul bulundu ({len(res1)} node)")
        else:
            print("   ⚠ İstanbul location bulunamadı")
        print()

        # 2) İstanbul'daki restoranlar
        q2 = """
        MATCH (r:Restaurant)-[:LOCATED_IN]->(l:Location)
        WHERE toLower(l.name) = 'istanbul'
        RETURN r.name as restoran
        LIMIT 10
        """
        res2 = list(session.run(q2))
        print("2) İstanbul'daki restoranlar:")
        if res2:
            for r in res2:
                print("   -", r["restoran"])
        else:
            print("   ⚠ Hiç restoran bulunamadı")
        print()

        # 3) Romantik occasion kontrolü
        q3 = """
        MATCH (o:Occasion)
        WHERE toLower(o.name) = 'romantik'
        RETURN o.name as name, o.id as id
        """
        res3 = list(session.run(q3))
        print("3) Occasion kontrolü:")
        if res3:
            print(f"   ✓ Romantik occasion bulundu ({len(res3)} node)")
        else:
            print("   ⚠ Romantik occasion bulunamadı")
        print()

        # 4) İstanbul + romantik restoran eşleşmesi
        q4 = """
        MATCH (r:Restaurant)-[:LOCATED_IN]->(l:Location),
              (r)-[:SUITS_OCCASION]->(o:Occasion)
        WHERE toLower(l.name) = 'istanbul'
          AND toLower(o.name) = 'romantik'
        RETURN r.name as restoran
        LIMIT 10
        """
        res4 = list(session.run(q4))
        print("4) İstanbul'da romantik restoranlar:")
        if res4:
            for r in res4:
                print("   -", r["restoran"])
        else:
            print("   ⚠ İstanbul'da romantik restoran bulunamadı")
        print()

        # 5) İstanbul'da TASTE rating alan restoranlar
        q5 = """
        MATCH (r:Restaurant)-[:LOCATED_IN]->(l:Location),
              (r)-[rating:RATED_FOR]->(a:Aspect)
        WHERE toLower(l.name) = 'istanbul'
          AND rating.dimension = 'TASTE'
        RETURN r.name as restoran, rating.sentiment as duygu, rating.score as puan
        LIMIT 10
        """
        res5 = list(session.run(q5))
        print("5) İstanbul'da TASTE puanı olan restoranlar:")
        if res5:
            for r in res5:
                print(f"   - {r['restoran']} | {r['duygu']} ({r['puan']})")
        else:
            print("   ⚠ İstanbul'da TASTE puanı olan restoran bulunamadı")
        print()

        # 6) İstanbul + romantik + positive taste kontrolü
        q6 = """
        MATCH (r:Restaurant)-[:LOCATED_IN]->(l:Location),
              (r)-[rating:RATED_FOR]->(a:Aspect),
              (r)-[:SUITS_OCCASION]->(o:Occasion)
        WHERE toLower(l.name) = 'istanbul'
          AND rating.dimension = 'TASTE'
          AND rating.sentiment = 'positive'
          AND toLower(o.name) = 'romantik'
        RETURN r.name as restoran, avg(rating.score) as ortalama_puan
        ORDER BY ortalama_puan DESC
        LIMIT 10
        """
        res6 = list(session.run(q6))
        print("6) İstanbul'da romantik + positive TASTE restoranlar:")
        if res6:
            for r in res6:
                print(f"   - {r['restoran']} | Ortalama puan: {r['ortalama_puan']}")
        else:
            print("   ⚠ Hiç eşleşme bulunamadı")
        print()

    print("✅ Debug tamamlandı.")


In [16]:
debug_istanbul_romantik_taste(kg)

🔎 Debug başlıyor...

1) Location kontrolü:
   ✓ İstanbul bulundu (1 node)

2) İstanbul'daki restoranlar:
   - babil_fastfoodev_yemekleri
   - quick_china_metropol_avm
   - beykoz_meydan_balik
   - tike_istmarina
   - divane_cihangir_yeni_nesil_meyhane
   - chin_chin_etiler
   - hayat_memat_balikcisi
   - caki_restoran
   - key_karakoy
   - antephan_kunefe

3) Occasion kontrolü:
   ✓ Romantik occasion bulundu (1 node)

4) İstanbul'da romantik restoranlar:
   - hemdem_steak_house
   - gazebo
   - izaka_terrace
   - le_vapeur_magique_tdi_karakoy_iskelesi
   - city_steak_house
   - grace_rooftop_restaurant
   - 16_roof_swissotel_the_bosphorus_istanbul
   - trend_216_cafe_restaurant
   - kasibeyaz_akvaryum
   - olive_anatolian_restaurant

5) İstanbul'da TASTE puanı olan restoranlar:
   - babil_fastfoodev_yemekleri | positive (0.875)
   - babil_fastfoodev_yemekleri | positive (0.925)
   - babil_fastfoodev_yemekleri | positive (0.85)
   - babil_fastfoodev_yemekleri | positive (0.875)
   - bab

## Cell 15: Kompleks Çoklu Relation Sorguları
Bu cell'de "Ankara Beypazarı'nda çocuk bahçesi olan kebapçı" tarzı kompleks sorgular için geliştirilmiş fonksiyonlar bulunur:

**Yeni özellikler:**
- `search_restaurants_multi_criteria()`: Şehir + ilçe + menü + amenity kombinasyonları
- `find_restaurants_with_all_criteria()`: Dictionary tabanlı esnek arama
- 4 farklı relation'a aynı anda bakabilen sorgular

**Desteklenen kriterler:**
- Konum: city, district (LOCATED_IN relation)  
- Menü: menu_contains (SERVES relation)
- Özellikler: amenities (HAS_AMENITY relation)
- Durumlar: occasions (SUITS_OCCASION relation)
- Puan: min_score (RATED_FOR relation)


In [17]:
# ========= 9) Kompleks Çoklu Relation Sorgu Fonksiyonları =========
def chatbot_query_restaurants(kg, criteria, limit=5):
    """
    Chatbot için restoran arama fonksiyonu.
    criteria dict örneği:
    {
        "city": "ankara",
        "district": "beypazarı",
        "menu": "kebap",
        "amenities": ["çocuk parkı"],
        "occasion": "romantik",
        "min_score": 0.7
    }
    """
    if kg is None:
        return [{"error": "Neo4j bağlantısı yok"}]
    
    with kg.driver.session() as session:
        match_clauses = ["(r:Restaurant)"]
        conditions = []
        params = {"limit": limit}
        
        # Şehir
        if "city" in criteria:
            match_clauses.append("(r)-[:LOCATED_IN]->(city:Location)")
            conditions.append("toLower(city.name) = $city")
            params["city"] = criteria["city"].lower()
        
        # İlçe
        if "district" in criteria:
            match_clauses.append("(r)-[:LOCATED_IN]->(district:Location)")
            conditions.append("toLower(district.name) = $district")
            params["district"] = criteria["district"].lower()
        
        # Menü
        if "menu" in criteria:
            match_clauses.append("(r)-[:SERVES]->(m:MenuItem)")
            conditions.append("toLower(m.name) CONTAINS $menu")
            params["menu"] = criteria["menu"].lower()
        
        # Amenity
        if "amenities" in criteria:
            for i, amenity in enumerate(criteria["amenities"]):
                match_clauses.append(f"(r)-[:HAS_AMENITY]->(a{i}:Amenity)")
                conditions.append(f"toLower(a{i}.name) CONTAINS $amenity{i}")
                params[f"amenity{i}"] = amenity.lower()
        
        # Occasion
        if "occasion" in criteria:
            match_clauses.append("(r)-[:SUITS_OCCASION]->(o:Occasion)")
            conditions.append("toLower(o.name) = $occasion")
            params["occasion"] = criteria["occasion"].lower()
        
        # Min score
        if "min_score" in criteria:
            match_clauses.append("(r)-[rating:RATED_FOR]->(asp:Aspect)")
            conditions.append("rating.score >= $min_score AND rating.sentiment = 'positive'")
            params["min_score"] = criteria["min_score"]
        
        match_clause = "MATCH " + ", ".join(match_clauses)
        where_clause = "WHERE " + " AND ".join(conditions) if conditions else ""
        
        query = f"""
        {match_clause}
        {where_clause}
        OPTIONAL MATCH (r)-[rt:RATED_FOR]->(asp:Aspect)
        OPTIONAL MATCH (r)-[:HAS_AMENITY]->(am:Amenity)
        OPTIONAL MATCH (r)-[:SERVES]->(m:MenuItem)
        OPTIONAL MATCH (r)-[:SUITS_OCCASION]->(o:Occasion)
        WHERE rt.sentiment = 'positive'
        WITH r, 
             avg(rt.score) as avg_score,
             collect(DISTINCT asp.name) as aspects,
             collect(DISTINCT am.name) as amenities,
             collect(DISTINCT m.name) as menu_items,
             collect(DISTINCT o.name) as occasions
        RETURN r.name as restoran,
               round(avg_score,2) as ortalama_puan,
               aspects, amenities, menu_items, occasions
        ORDER BY ortalama_puan DESC
        LIMIT $limit
        """
        
        result = session.run(query, params)
        return [dict(record) for record in result]



In [18]:
criteria = {
    "city": "İstanbul",
    "occasion": "romantik",
    "menu": "kebap",
    "min_score": 0.7
}
cevap = chatbot_query_restaurants(kg, criteria)
print(cevap)

[{'restoran': 'roof_mezze_360_restaurant', 'ortalama_puan': 0.86, 'aspects': ['manzara', 'tatlılar', 'nazik personel', 'profesyonel personel', 'canlı müzik', 'ilgili personel', 'çeşitli menü', 'mezeler', 'iyi fiyat-performans', 'samimi atmosfer', 'gün batımı manzarası', 'kuzu pirzola', 'özel gün ortamı', 'glutensiz seçenekler', 'ana yemekler', 'rahatlatıcı atmosfer', 'romantik ortam'], 'amenities': ['canlı müzik'], 'menu_items': ['kebap'], 'occasions': ['romantik']}, {'restoran': 'papuli_restaurant', 'ortalama_puan': 0.85, 'aspects': ['manzara', 'tuvalet bakımsızlığı', 'atmosfer', 'güleryüzlü personel', 'canlı müzik', 'tuvalet temizliği', 'göl manzarası', 'mezeler', 'ilgili garsonlar', 'soğan sarması', 'çiğdem hanım ın performansı', 'gün batımı'], 'amenities': ['canlı müzik'], 'menu_items': ['adana kebap'], 'occasions': ['romantik']}, {'restoran': 'arca_restaurant_grill', 'ortalama_puan': 0.84, 'aspects': ['manzara', 'şık ambiyans', 'hızlı servis', 'ilgili personel', 'güler yüzlü perso

In [19]:
def format_chatbot_response(results):
    if not results:
        return "⚠ Hiç uygun mekan bulunamadı."
    
    cevap = ["✨ Önerilen mekanlar:"]
    for i, r in enumerate(results, 1):
        line = f"{i}. {r['restoran']} (Puan: {r['ortalama_puan']})"
        
        # Aspects
        if r.get("aspects"):
            line += f" | Öne çıkan: {', '.join(r['aspects'][:3])}"
        
        # Eğer amenity varsa göster
        if r.get("amenities"):
            line += f" | İmkanlar: {', '.join(r['amenities'])}"
        
        # Eğer occasion varsa göster
        if r.get("occasions"):
            line += f" | Uygun: {', '.join(r['occasions'])}"
        
        # Eğer menu varsa göster
        if r.get("menu_items"):
            line += f" | Menü: {', '.join(r['menu_items'][:3])}"
        
        cevap.append(line)
    return "\n".join(cevap)


## Cell 16: Kompleks Sorgu Test Örnekleri
Bu cell'de çoklu relation sorguları test edilir. "Ankara Beypazarı'nda çocuk bahçesi olan kebapçı" tarzı kompleks arama kriterleri denenecek.


In [ ]:
# ========= 10) Chatbot Query Testleri =========

print("🎯 CHATBOT TABANLI ÇOKLU RELATION TESTLERİ")
print("=" * 60)

if kg is None:
    print("❌ Neo4j bağlantısı yok!")
else:
    print("✅ Neo4j bağlantısı aktif")

# Test 1: Ankara Beypazarı'nda çocuk parkı olan kebapçı
print("\n🔥 TEST 1: Ankara Beypazarı'nda çocuk parkı olan kebapçı")
criteria1 = {
    "city": "ankara",
    "district": "beypazarı",
    "menu": "kebap",
    "amenities": ["çocuk parkı"]
}
results1 = chatbot_query_restaurants(kg, criteria1, limit=5) if kg else []
if results1:
    print(format_chatbot_response(results1))
else:
    print("❌ Bu kriterlere uygun restoran bulunamadı.")
    print("💡 Daha az kriter ile tekrar deneyin.")

# Test 2: Ankara'da bahçeli restoranlar
print("\n🔥 TEST 2: Ankara'da bahçeli restoranlar")
criteria2 = {
    "city": "ankara",
    "amenities": ["bahçe"]
}
results2 = chatbot_query_restaurants(kg, criteria2, limit=5) if kg else []
if results2:
    print(format_chatbot_response(results2))
else:
    print("❌ Ankara'da bahçeli restoran bulunamadı.")

# Test 3: İzmir Konak'ta pizza mekanı
print("\n🔥 TEST 3: İzmir Konak'ta pizza mekanı")
criteria3 = {
    "city": "izmir",
    "district": "konak",
    "menu": "pizza"
}
results3 = chatbot_query_restaurants(kg, criteria3, limit=5) if kg else []
if results3:
    print(format_chatbot_response(results3))
else:
    print("❌ İzmir Konak'ta pizza mekanı bulunamadı.")

# Test 4: İstanbul'da romantik kebapçı
print("\n🔥 TEST 4: İstanbul'da romantik kebapçı")
criteria4 = {
    "city": "istanbul",
    "menu": "kebap",
    "occasion": "romantik"
}
results4 = chatbot_query_restaurants(kg, criteria4, limit=5) if kg else []
if results4:
    print(format_chatbot_response(results4))
else:
    print("❌ İstanbul'da romantitaşk kebapçı bulunamadı.")

# Test 5: İzmir'de yüksek puanlı kahvaltıcılar
print("\n🔥 TEST 5: İzmir'de yüksek puanlı pizzacılar")
criteria5 = {
    "city": "izmir",
    "menu": "pizza",
    "min_score": 0.8
}
results5 = chatbot_query_restaurants(kg, criteria5, limit=5) if kg else []
if results5:
    print(format_chatbot_response(results5))
else:
    print("❌ İzmir'de yüksek puanlı kahvaltıcı bulunamadı.")


🎯 CHATBOT TABANLI ÇOKLU RELATION TESTLERİ
✅ Neo4j bağlantısı aktif

🔥 TEST 1: Ankara Beypazarı'nda çocuk parkı olan kebapçı
❌ Bu kriterlere uygun restoran bulunamadı.
💡 Daha az kriter ile tekrar deneyin.

🔥 TEST 2: Ankara'da bahçeli restoranlar
✨ Önerilen mekanlar:
1. koy_bahcesi (Puan: 0.88) | Öne çıkan: uzun bekleme süresi, ilgisiz personel, yavaş servis | İmkanlar: bahçe | Menü: lezzetli pide, lezzetli pizza, vasat hamburger
2. isirgan_sirdan_mumbar_kelle (Puan: 0.87) | Öne çıkan: kelle paça, geleneksel yemekler, mumbar | İmkanlar: bahçe | Uygun: aile | Menü: küçük lahmacun
3. marrakesh_cafe (Puan: 0.87) | Öne çıkan: kahvaltı, ilgisiz garsonlar, güzel atmosfer | İmkanlar: bahçe | Uygun: aile
4. nefis_bahce (Puan: 0.87) | Öne çıkan: yüksek fiyat, huzurlu, pahalı fiyat | İmkanlar: bahçe
5. mutlu_kebap (Puan: 0.86) | Öne çıkan: hizmet kalitesinde düşüş, huzurlu ortam, ağaçlı bahçe | İmkanlar: bahçe

🔥 TEST 3: İzmir Konak'ta pizza mekanı
✨ Önerilen mekanlar:
1. la_fourmi_restaurant (Pua

In [21]:
def summarize_test_results(test_results):
    """
    Çoklu chatbot test sonuçlarını tablo formatında özetler.
    test_results: list of tuples (test_name, results)
    """
    summary = []
    for test_name, results in test_results:
        if not results:
            summary.append({
                "Test": test_name,
                "Durum": "❌ Sonuç yok",
                "Mekan Sayısı": 0,
                "Örnek": "-"
            })
        else:
            summary.append({
                "Test": test_name,
                "Durum": "✅ Bulundu",
                "Mekan Sayısı": len(results),
                "Örnek": results[0]['restoran']
            })
    
    return pd.DataFrame(summary)


In [22]:
from IPython.display import display
test_results = [
    ("Ankara Beypazarı çocuk parkı + kebap", results1),
    ("Ankara bahçeli", results2),
    ("İzmir Konak pizza", results3),
    ("İstanbul romantik kebapçı", results4),
    ("İzmir yüksek puanlı kahvaltıcılar", results5),
]

df_summary = summarize_test_results(test_results)
display(df_summary)


,Test,Durum,Mekan Sayısı,Örnek
0,Ankara Beypazarı çocuk parkı + kebap,❌ Sonuç yok,0,-
1,Ankara bahçeli,✅ Bulundu,5,koy_bahcesi
2,İzmir Konak pizza,✅ Bulundu,2,la_fourmi_restaurant
3,İstanbul romantik kebapçı,✅ Bulundu,5,roof_mezze_360_restaurant
4,İzmir yüksek puanlı kahvaltıcılar,❌ Sonuç yok,0,-


In [57]:
# Dinamik listeler edges_df üzerinden
CITY_LIST = set(
    edges_df.loc[edges_df["relation"]=="LOCATED_IN"]
    .apply(lambda row: row["props"].get("name") if row["props"].get("level")=="city" else None, axis=1)
    .dropna()
    .unique()
)

DISTRICT_LIST = {}
for city in CITY_LIST:
    districts = edges_df.loc[
        (edges_df["relation"]=="LOCATED_IN") & 
        (edges_df["props"].apply(lambda p: p.get("level")=="district")) &
        (edges_df["head"].isin(
            edges_df.loc[
                (edges_df["relation"]=="LOCATED_IN") &
                (edges_df["props"].apply(lambda p: p.get("level")=="city")) &
                (edges_df["props"].apply(lambda p: p.get("name")==city)),
                "head"
            ]
        ))
    ]["props"].apply(lambda p: p.get("name")).unique()
    if len(districts)>0:
        DISTRICT_LIST[city] = list(districts)

AMENITIES = set(
    edges_df.loc[edges_df["relation"]=="HAS_AMENITY","tail"]
    .str.replace("amenity:","")
    .unique()
)

OCCASIONS = set(
    edges_df.loc[edges_df["relation"]=="SUITS_OCCASION","tail"]
    .str.replace("occasion:","")
    .unique()
)

MENU_ITEMS = set(
    edges_df.loc[edges_df["relation"]=="SERVES","tail"]
    .str.replace("menu:","")
    .unique()
)

In [59]:
def parse_user_query(query: str):
    """Doğal dil sorgusunu criteria dict'e çevirir"""
    q = norm(query)  # normalize: küçük harf + türkçe harf düzeltme
    criteria = {}
    
    # City
    for city in CITY_LIST:
        if city in q:
            criteria["city"] = city
            break  # tek şehir yeterli
    
    # District
    for city, districts in DISTRICT_LIST.items():
        for d in districts:
            if d in q:
                criteria["district"] = d
                # City belirtilmemişse district'ten city çıkar
                if "city" not in criteria:
                    criteria["city"] = city
                break
    
    # Amenities
    found_amenities = [a for a in AMENITIES if a in q]
    if found_amenities:
        criteria["amenities"] = found_amenities
    
    # Occasion
    for occ in OCCASIONS:
        if occ in q:
            criteria["occasion"] = occ
            break
    
    # Menu items
    for item in MENU_ITEMS:
        if item in q:
            criteria["menu"] = item
            break
    
    # Min score (örnek: 'yüksek puanlı', 'en iyi', 'iyi mekan' → min_score=0.7)
    if "yüksek puan" in q or "en iyi" in q or "iyi " in q:
        criteria["min_score"] = 0.7
    
    return criteria

In [60]:
user_q = "izmir'de kebapçı"
criteria = parse_user_query(user_q)
results = chatbot_query_restaurants(kg, criteria, limit=5)
print(format_chatbot_response(results))


✨ Önerilen mekanlar:
1. lokanta_biber_ocakbasi_ve_meyhane (Puan: 0.88) | Öne çıkan: uygun fiyatlı, ciğer şiş, güleryüzlü hizmet | Menü: adana kebap
2. gencbey_restaurant (Puan: 0.87) | Öne çıkan: çorbalar, temizlik, kahvaltı seçenekleri | İmkanlar: bahçe | Menü: adana kebap
3. konyali_alsancak (Puan: 0.87) | Öne çıkan: çorbalar, iskender kebabı, ikramlar | İmkanlar: otopark | Menü: adana kebap
4. damla (Puan: 0.86) | Öne çıkan: kokoreç, lezzetli mezeler, servis kalitesi düşüşü | Menü: kuzu kebap
5. can_can_kardesler (Puan: 0.84) | Öne çıkan: çorbalar, temizlik, güler yüzlü çalışanlar | Menü: kebap


In [61]:
print(parse_user_query("İstanbul'da romantik kebapçı"))

print(parse_user_query("İzmir Beypazarı'nda çocuk parkı olan kebapçı"))
# {'city': 'ankara', 'district': 'beypazarı', 'menu': 'kebap', 'amenities': ['çocuk parkı']}

print(parse_user_query("İzmir'de Konak'ta pizza mekanı"))
# {'city': 'izmir', 'district': 'konak', 'menu': 'pizza'}

print(parse_user_query("Ankara'da bahçeli restoran"))
# {'city': 'ankara', 'amenities': ['bahçe']}

print(parse_user_query("İstanbulda yüksek puanlı pizzacılar"))
# {'city': 'izmir', 'menu': 'pizza', 'min_score': 0.7}


{'city': 'istanbul', 'occasion': 'romantik', 'menu': 'kebap'}
{'city': 'izmir', 'district': 'beypazarı', 'amenities': ['çocuk parkı'], 'menu': 'kebap'}
{'city': 'izmir', 'district': 'konak', 'menu': 'pizza'}
{'city': 'ankara', 'amenities': ['bahçe']}
{'city': 'istanbul', 'menu': 'pizza', 'min_score': 0.7}


In [62]:
def chatbot_loop(kg):
    print("🤖 Restoran Chatbot'a hoş geldiniz!")
    print("Çıkmak için 'exit' yazın.\n")
    
    while True:
        user_q = input("👤 Siz: ")
        if user_q.lower().strip() in ["exit", "quit", "çıkış", "q"]:
            print("👋 Görüşmek üzere!")
            break
        
        # Kullanıcı sorgusunu parse et
        criteria = parse_user_query(user_q)
        print(f"🔎 Algılanan kriterler: {criteria}")
        
        # Graph'tan sonuçları çek
        results = chatbot_query_restaurants(kg, criteria, limit=5)
        
        # Cevabı formatla ve yazdır
        cevap = format_chatbot_response(results)
        print(f"\n🤖 Chatbot:\n{cevap}\n")

chatbot_loop(kg)

🤖 Restoran Chatbot'a hoş geldiniz!
Çıkmak için 'exit' yazın.

🔎 Algılanan kriterler: {'city': 'izmir', 'district': 'konak', 'menu': 'pizza'}

🤖 Chatbot:
✨ Önerilen mekanlar:
1. la_fourmi_restaurant (Puan: 0.84) | Öne çıkan: düşük servis kalitesi, makarna, atmosfer | İmkanlar: canlı müzik | Uygun: romantik | Menü: pizza
2. favino_alsancak (Puan: 0.81) | Öne çıkan: makarna, hızlı servis, uygun fiyatlar | Uygun: romantik | Menü: pizza

🔎 Algılanan kriterler: {'city': 'izmir', 'amenities': ['bahçe']}

🤖 Chatbot:
✨ Önerilen mekanlar:
1. das_cantina (Puan: 0.88) | Öne çıkan: yüksek fiyat, lezzetli, küçük porsiyon | İmkanlar: teras, bahçe | Menü: baja usulü levrek taco, çıtır tavuk taco, karidesli taco
2. gencbey_restaurant (Puan: 0.87) | Öne çıkan: çorbalar, temizlik, kahvaltı seçenekleri | İmkanlar: bahçe | Menü: adana kebap
3. antik_koy (Puan: 0.86) | Öne çıkan: ilgisiz garsonlar, sipariş gecikmeleri, yüksek yemek fiyatları | İmkanlar: bahçe | Uygun: aile
4. oteyaka (Puan: 0.86) | Öne çıka